In [19]:
!pip install xlrd
!pip install openpyxl
!pip install pandas

In [28]:
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import re

In [29]:
xlsx = pd.read_excel("raw-data.xlsx")
xlsx.to_csv("transformed-data.csv")

In [30]:
raw_data = pd.read_csv('transformed-data.csv')
print(raw_data['content.1'])

0                                                    수신 불가
1        안녕하세요!시스템통합운영센터 서비스데스크입니다. 네이버, 다음, 구글, 네이트 + ...
2        안녕하세요!시스템통합운영센터 서비스데스크입니다. 요청해주신 주소로 세금계산서 정보 ...
3        안녕하세요!시스템통합운영센터 서비스데스크입니다. 요청 주신 내용 유선으로 안내드렸습...
4        안녕하세요...위 설정작업후 mysql 재시작 부탁드립니다. (현재 5.6버전입니다...
                               ...                        
75091    네임서버 후이즈로 변경을 했습니다.저희도 업체에 요청을 해야 해서 업체에 요청하여 ...
75092    salesnut.kr1. SSL 인증서 설치 요청 . https 연결 요청 드립니다...
75093    안녕하세요!디딤365 운영센터입니다.현재 도메인 연결은 완료하였으며, 사이트 확인 ...
75094    제가 잘못썻네요 SSH (https) 이 아니라 SSL (https) 인증서 작업 ...
75095    안녕하세요!디딤365 운영센터입니다.유선상 말씀드린 경로 안내해 드립니다.네임 서버...
Name: content.1, Length: 75096, dtype: object


In [31]:
new_data = pd.DataFrame({'data_idx':[], 'Question':[], 'Answer':[]})
#new_data = new_data.append(pd.Series(['1', '질문', '답변'], index=['data_idx', 'Question', 'Answer']), ignore_index=True)
data_idx = None
question = ''
answer = ''
for idx, row in tqdm_notebook(raw_data.iterrows()):
  #새로운 질문이 시작됐거나 질문과 답변이 모두 찼는데 새로운 질문이 들어온 경우
  if data_idx != row['data_idx'] or (question != '' and answer != '' and row['writer'] == '고객'):
    if data_idx != None: #새로운 질문이 들어오면 기존에 받아놨던 데이터 테이블에 올리기
      new_data = new_data.append(pd.Series([data_idx, question, answer], index=['data_idx', 'Question', 'Answer']), ignore_index=True)
    data_idx = row['data_idx']
    question = row['content']
    answer = ''
  if row['writer'] == '고객':
    question = question + row['content.1']
  if(row['writer'] == '디딤365'):
    answer = answer + row['content.1']
#마지막 데이터 넣어주기
new_data = new_data.append(pd.Series([data_idx, question, answer], index=['data_idx', 'Question', 'Answer']), ignore_index=True)

print(new_data)
new_data.to_csv("beta-dataset.csv", index=False, encoding="utf-8-sig")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



                data_idx  ...                                             Answer
0      M-0001-10621       ...  안녕하세요!시스템통합운영센터 서비스데스크입니다. 네이버, 다음, 구글, 네이트 + ...
1      M-0001-10626       ...  안녕하세요!시스템통합운영센터 서비스데스크입니다. 요청해주신 주소로 세금계산서 정보 ...
2      M-0001-10636       ...  안녕하세요!시스템통합운영센터 서비스데스크입니다. 요청 주신 내용 유선으로 안내드렸습...
3      M-0001-10673       ...  안녕하세요!시스템통합운영센터 서비스데스크입니다. 요청하신 설정 적용 후 MySQL ...
4      M-0001-10690       ...  안녕하세요!시스템통합운영센터 서비스데스크입니다. 요청하신 작업 일시에 증설 진행하겠...
...                  ...  ...                                                ...
49253          P-52995    ...  안녕하세요!디딤365 운영센터입니다.문의하신 사항에 대해 안내해 드립니다.어제 서버...
49254          P-53006    ...  안녕하세요!디딤365 영업컨설팅본부 장윤정 입니다.요청주신 리눅스 웹 호스팅 견적서...
49255          P-53037    ...  안녕하세요!디딤365 운영센터입니다.네임서버의 경우 후이즈에 DNS를 세팅하셔도 상...
49256          P-53037    ...  안녕하세요!디딤365 운영센터입니다.현재 도메인 연결은 완료하였으며, 사이트 확인 ...
49257          P-53037    ...  안녕하세요!디딤365 운영센터입니다.유선상 말씀드린 경로 안내해 드립니다.네임 서버...

[49258 rows x 3 columns]
